In [37]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import geopandas as gpd
import os 
import sys

In [38]:
df = pd.read_csv('../data/plant_data.csv')

/tmp/ipykernel_18875/3742021595.py:1: DtypeWarning: Columns (0,1,4,6,8,16,17,19,20,22,23,27,29,33,34,35,50,51,52,53,55,56,58,59,60,61,62,63,64,65,66,67,68,69,71,72,74,82,85,86,97,98,99,102,107,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/plant_data.csv')


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12613 entries, 0 to 12612
Columns: 150 entries, Plant file sequence number to Plant total noncombustion other unknown/purchased generation percent (resource mix)
dtypes: object(150)
memory usage: 14.4+ MB


In [40]:
print(len(df))

12613


In [41]:
df.head()

,Plant file sequence number,Data Year,Plant state abbreviation,Plant name,DOE/EIA ORIS plant or facility code,Plant transmission or distribution system owner name,Plant transmission or distribution system owner ID,Utility name,Utility ID,Plant-level sector,...,Plant geothermal generation percent (resource mix),Plant other fossil generation percent (resource mix),Plant other unknown / purchased fuel generation percent (resource mix),Plant total nonrenewables generation percent (resource mix),Plant total renewables generation percent (resource mix),Plant total nonrenewables other unknown/purchased generation percent (resource mix),Plant total nonhydro renewables generation percent (resource mix),Plant total combustion generation percent (resource mix),Plant total noncombustion generation percent (resource mix),Plant total noncombustion other unknown/purchased generation percent (resource mix)
0,SEQPLT23,YEAR,PSTATABB,PNAME,ORISPL,OPRNAME,OPRCODE,UTLSRVNM,UTLSRVID,SECTOR,...,PLGTPR,PLOFPR,PLOPPR,PLTNPR,PLTRPR,PLTOPR,PLTHPR,PLCYPR,PLCNPR,PLCOPR
1,1,2023,AK,Alakanuk,57053,"Alaska Village Elec Coop, Inc",221,"Alaska Village Elec Coop, Inc",221,Electric Utility,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2023,AK,Allison Creek Hydro,58982,"Copper Valley Elec Assn, Inc",4329,"Copper Valley Elec Assn, Inc",4329,Electric Utility,...,0.0%,0.0%,0.0%,0.0%,100.0%,0.0%,0.0%,0.0%,100.0%,0.0%
3,3,2023,AK,Ambler,60243,"Alaska Village Elec Coop, Inc",221,"Alaska Village Elec Coop, Inc",221,Electric Utility,...,0.0%,0.0%,0.0%,100.0%,0.0%,0.0%,0.0%,100.0%,0.0%,0.0%
4,4,2023,AK,Angoon,7462,"Inside Passage Elec Coop, Inc",18963,"Inside Passage Elec Coop, Inc",18963,Electric Utility,...,0.0%,0.0%,0.0%,100.0%,0.0%,0.0%,0.0%,100.0%,0.0%,0.0%


In [42]:

for i in df.columns:
    print(i)

Plant file sequence number
Data Year
Plant state abbreviation
Plant name
DOE/EIA ORIS plant or facility code
Plant transmission or distribution system owner name
Plant transmission or distribution system owner ID
Utility name
Utility ID
Plant-level sector
Balancing Authority Name
Balancing Authority Code
NERC region acronym
eGRID subregion acronym
eGRID subregion name
Plant associated ISO/RTO Territory
Plant FIPS state code
Plant FIPS county code
Plant county name
Plant latitude
Plant longitude
CAPD Program Flag
Number of units
Number of generators
Plant primary fuel
Plant primary fuel category
Flag indicating if the plant burned or generated any amount of coal
Plant capacity factor
Plant nameplate capacity (MW)
Nonbaseload Factor
Biogas/ biomass plant adjustment flag
Combined heat and power (CHP) plant adjustment flag
CHP plant useful thermal output (MMBtu)
CHP plant power to heat ratio
CHP plant electric allocation factor
Plant pumped storage flag
Plant annual heat input from combust

Energy Resources
- Plant annual coal net generation (MWh) 
- Plant annual oil net generation (MWh)
- Plant annual gas net generation (MWh)
- Plant annual nuclear net generation (MWh)
- Plant annual hydro net generation (MWh)
- Plant annual biomass net generation (MWh)
- Plant annual wind net generation (MWh)
- Plant annual solar net generation (MWh)
- Plant annual geothermal net generation (MWh)

In [43]:
def resource_fgdf(resource_col):

    lon_col  = "Plant longitude"
    lat_col  = "Plant latitude"

    cols_needed = [lon_col, lat_col, resource_col]
    df = df[cols_needed].dropna(subset=[lon_col, lat_col]).copy()



    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs="EPSG:4326"
    )

    return gdf



In [44]:
lon_col  = "Plant longitude"
lat_col  = "Plant latitude"
coal_col = "Plant annual coal net generation (MWh)"
oil_col  = "Plant annual oil net generation (MWh)"

# Keep just what we need; drop rows without coordinates
cols_needed = [lon_col, lat_col, coal_col, oil_col]
df = df[cols_needed].dropna(subset=[lon_col, lat_col]).copy()

# # Ensure numeric
for c in df.columns:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)


gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
    crs="EPSG:4326"
)

In [45]:
def robust_norm(series, q=0.95):
    top = series.quantile(q)
    return np.clip(series / top, 0, 1) if top > 0 else series*0

gdf["coal_n"] = robust_norm(gdf[coal_col])
gdf["oil_n"]  = robust_norm(gdf[oil_col])